In [3]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [4]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [5]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

In [6]:
import numpy as np
##SELECT 25% OF USERS WITH LONGEST PROFILES TO FINE-TUNE THE HYPERPARAMETERS
users_to_ignore=[]
profile_length = np.ediff1d(URM_train.indptr)
block_size = int(len(profile_length) * 0.25)

start_pos = 3 * block_size
end_pos = min(4 * block_size, len(profile_length))
sorted_users = np.argsort(profile_length)

users_in_group = sorted_users[start_pos:end_pos]

users_in_group_p_len = profile_length[users_in_group]
sorted_users = np.argsort(profile_length)

users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)

In [12]:
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10],ignore_users=sorted_users[users_not_in_group_flag])

EvaluatorHoldout: Ignoring 13920 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13905 ( 0.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10488 Users


In [5]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [7]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 1 ( 0.0%) users with no interactions.


In [8]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18000/18059 [13:16<00:01, 39.98it/s]

In [9]:
similarity_slime = SLIME_recommender.W_sparse
similarity_rp3 = RP3Beta_recommender.W_sparse

In [10]:

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = URM_stacked,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=494, alpha=0.1597863887003789 )

In [11]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 11.35 sec


100%|████████████████████████████████████▉| 18056/18059 [13:30<00:00, 39.98it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 21.19 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 31.32 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 42.83 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 53.15 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 1.06 min
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.25 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.42 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.57 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.71 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.71 min


In [21]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.4, 1.0),
    "beta": Real(0.1, 0.3)
}
recommenders = [similarity_hybrid, IALS_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7518478555465858, 'beta': 0.29400336161369167}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 3496 (100.0%) in 4.65 sec. Users per second: 752
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.7518478555465858, 'beta': 0.29400336161369167} - results: PRECISION: 0.0779748, PRECISION_RECALL_MIN_DEN: 0.0779748, RECALL: 0.0088197, MAP: 0.0276603, MAP_MIN_DEN: 0.0276603, MRR: 0.1936583, NDCG: 0.0786251, F1: 0.0158469, HIT_RATE: 0.5557780, ARHR_ALL_HITS: 0.2316480, NOVELTY: 0.0056753, AVERAGE_POPULARITY: 0.4739279, DIVERSITY_MEAN_INTER_LIST: 0.9474009, DIVERSITY_HERFINDAHL: 0.9947130, COVERAGE_ITEM: 0.0598040, COVERAGE_ITEM_CORRECT: 0.0305665, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.5557780, DIVERSITY_GINI: 0.0145790, SHANNON_ENTROPY: 8.3899941, RATIO_DIVERSITY_HERFINDAHL: 0.9950800, RATIO_DIVERSITY_GI

SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.8383656986063074, 'beta': 0.13037836321941995} - results:
CUTOFF: 10 - PRECISION: 0.0804095, PRECISION_RECALL_MIN_DEN: 0.0816101, RECALL: 0.0197973, MAP: 0.0293437, MAP_MIN_DEN: 0.0297499, MRR: 0.2015176, NDCG: 0.0824410, F1: 0.0317721, HIT_RATE: 0.5530891, ARHR_ALL_HITS: 0.2431581, NOVELTY: 0.0056658, AVERAGE_POPULARITY: 0.4993070, DIVERSITY_MEAN_INTER_LIST: 0.9257518, DIVERSITY_HERFINDAHL: 0.9925685, COVERAGE_ITEM: 0.1618030, COVERAGE_ITEM_CORRECT: 0.0555955, COVERAGE_USER: 0.9954233, COVERAGE_USER_CORRECT: 0.5505578, DIVERSITY_GINI: 0.0172747, SHANNON_ENTROPY: 8.3459305, RATIO_DIVERSITY_HERFINDAHL: 0.9929347, RATIO_DIVERSITY_GINI: 0.0646625, RATIO_SHANNON_ENTROPY: 0.6687366, RATIO_AVERAGE_POPULARITY: 2.5470588, RATIO_NOVELTY: 0.0299210, 


SearchBayesianSkopt: Saving model in logs/MergeTwoModelsByScore

MergeTwoModelsByScore: Saving model in file 'logs/MergeTwoModelsByScore_best_model'
MergeTwoModelsBySco

EvaluatorHoldout: Processed 3496 (100.0%) in 4.35 sec. Users per second: 804
SearchBayesianSkopt: Config 9 is suboptimal. Config: {'alpha': 0.5085378240990759, 'beta': 0.1858945508480087} - results: PRECISION: 0.0780892, PRECISION_RECALL_MIN_DEN: 0.0780892, RECALL: 0.0088261, MAP: 0.0277289, MAP_MIN_DEN: 0.0277289, MRR: 0.1938984, NDCG: 0.0787668, F1: 0.0158596, HIT_RATE: 0.5549199, ARHR_ALL_HITS: 0.2321413, NOVELTY: 0.0056749, AVERAGE_POPULARITY: 0.4745318, DIVERSITY_MEAN_INTER_LIST: 0.9470558, DIVERSITY_HERFINDAHL: 0.9946785, COVERAGE_ITEM: 0.0601916, COVERAGE_ITEM_CORRECT: 0.0306219, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.5549199, DIVERSITY_GINI: 0.0145886, SHANNON_ENTROPY: 8.3881462, RATIO_DIVERSITY_HERFINDAHL: 0.9950454, RATIO_DIVERSITY_GINI: 0.0546080, RATIO_SHANNON_ENTROPY: 0.6721192, RATIO_AVERAGE_POPULARITY: 2.4206763, RATIO_NOVELTY: 0.0299690, 

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 4.4645
Function value obtained: -0.0277
Current min

EvaluatorHoldout: Processed 3496 (100.0%) in 4.54 sec. Users per second: 770
SearchBayesianSkopt: Config 15 is suboptimal. Config: {'alpha': 0.44012321747663213, 'beta': 0.2115921179037274} - results: PRECISION: 0.0779176, PRECISION_RECALL_MIN_DEN: 0.0779176, RECALL: 0.0088164, MAP: 0.0276183, MAP_MIN_DEN: 0.0276183, MRR: 0.1933474, NDCG: 0.0785099, F1: 0.0158404, HIT_RATE: 0.5537757, ARHR_ALL_HITS: 0.2311367, NOVELTY: 0.0056778, AVERAGE_POPULARITY: 0.4714151, DIVERSITY_MEAN_INTER_LIST: 0.9487708, DIVERSITY_HERFINDAHL: 0.9948499, COVERAGE_ITEM: 0.0586965, COVERAGE_ITEM_CORRECT: 0.0305665, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.5537757, DIVERSITY_GINI: 0.0145800, SHANNON_ENTROPY: 8.3997427, RATIO_DIVERSITY_HERFINDAHL: 0.9952170, RATIO_DIVERSITY_GINI: 0.0545758, RATIO_SHANNON_ENTROPY: 0.6730484, RATIO_AVERAGE_POPULARITY: 2.4047772, RATIO_NOVELTY: 0.0299844, 

Iteration No: 16 ended. Evaluation done at random point.
Time taken: 4.6653
Function value obtained: -0.0276
Current m

EvaluatorHoldout: Processed 3496 (100.0%) in 4.81 sec. Users per second: 727
SearchBayesianSkopt: Config 22 is suboptimal. Config: {'alpha': 0.5600625221271265, 'beta': 0.12989603343518608} - results: PRECISION: 0.0791762, PRECISION_RECALL_MIN_DEN: 0.0791762, RECALL: 0.0089448, MAP: 0.0281380, MAP_MIN_DEN: 0.0281380, MRR: 0.1954507, NDCG: 0.0797359, F1: 0.0160737, HIT_RATE: 0.5563501, ARHR_ALL_HITS: 0.2346980, NOVELTY: 0.0056746, AVERAGE_POPULARITY: 0.4775237, DIVERSITY_MEAN_INTER_LIST: 0.9450089, DIVERSITY_HERFINDAHL: 0.9944739, COVERAGE_ITEM: 0.0626281, COVERAGE_ITEM_CORRECT: 0.0310648, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.5563501, DIVERSITY_GINI: 0.0147300, SHANNON_ENTROPY: 8.3829917, RATIO_DIVERSITY_HERFINDAHL: 0.9948407, RATIO_DIVERSITY_GINI: 0.0551372, RATIO_SHANNON_ENTROPY: 0.6717062, RATIO_AVERAGE_POPULARITY: 2.4359384, RATIO_NOVELTY: 0.0299673, 

Iteration No: 23 ended. Evaluation done at random point.
Time taken: 4.9414
Function value obtained: -0.0281
Current m

...

Iteration No: 173 ended. Search finished for the next optimal point.
Time taken: 7.2394
Function value obtained: -0.0279
Current minimum: -0.0283
Iteration No: 174 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7094058455823762, 'beta': 0.10268577557122825}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 3496 (100.0%) in 4.60 sec. Users per second: 760
SearchBayesianSkopt: Config 173 is suboptimal. Config: {'alpha': 0.7094058455823762, 'beta': 0.10268577557122825} - results: PRECISION: 0.0792048, PRECISION_RECALL_MIN_DEN: 0.0792048, RECALL: 0.0089455, MAP: 0.0282941, MAP_MIN_DEN: 0.0282941, MRR: 0.1968994, NDCG: 0.0800352, F1: 0.0160754, HIT_RATE: 0.5577803, ARHR_ALL_HITS: 0.2362202, NOVELTY: 0.0056778, AVERAGE_POPULARITY: 0.4787295, DIVERSITY_MEAN_INTER_LIST: 0.9437432, DIVERSITY_HERFINDAHL: 0.9943473, COVERAGE_ITEM: 0.0651753, COVERAGE_ITEM_CORRECT: 0.0318401, COVERAGE_USER: 1.00000

Iteration No: 180 ended. Search finished for the next optimal point.
Time taken: 7.5383
Function value obtained: -0.0283
Current minimum: -0.0283
Iteration No: 181 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 1.0, 'beta': 0.14035918633561698}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 3496 (100.0%) in 4.81 sec. Users per second: 727
SearchBayesianSkopt: Config 180 is suboptimal. Config: {'alpha': 1.0, 'beta': 0.14035918633561698} - results: PRECISION: 0.0792048, PRECISION_RECALL_MIN_DEN: 0.0792048, RECALL: 0.0089507, MAP: 0.0283229, MAP_MIN_DEN: 0.0283229, MRR: 0.1968835, NDCG: 0.0800500, F1: 0.0160838, HIT_RATE: 0.5574943, ARHR_ALL_HITS: 0.2363028, NOVELTY: 0.0056780, AVERAGE_POPULARITY: 0.4787495, DIVERSITY_MEAN_INTER_LIST: 0.9436984, DIVERSITY_HERFINDAHL: 0.9943428, COVERAGE_ITEM: 0.0651753, COVERAGE_ITEM_CORRECT: 0.0318401, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.5

Iteration No: 186 ended. Search finished for the next optimal point.
Time taken: 7.8665
Function value obtained: -0.0281
Current minimum: -0.0283
Iteration No: 187 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 1.0, 'beta': 0.16311219306707186}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 3496 (100.0%) in 4.98 sec. Users per second: 702
SearchBayesianSkopt: Config 186 is suboptimal. Config: {'alpha': 1.0, 'beta': 0.16311219306707186} - results: PRECISION: 0.0792048, PRECISION_RECALL_MIN_DEN: 0.0792048, RECALL: 0.0089365, MAP: 0.0281423, MAP_MIN_DEN: 0.0281423, MRR: 0.1953854, NDCG: 0.0797779, F1: 0.0160609, HIT_RATE: 0.5569222, ARHR_ALL_HITS: 0.2346787, NOVELTY: 0.0056760, AVERAGE_POPULARITY: 0.4788409, DIVERSITY_MEAN_INTER_LIST: 0.9439040, DIVERSITY_HERFINDAHL: 0.9943634, COVERAGE_ITEM: 0.0644000, COVERAGE_ITEM_CORRECT: 0.0319508, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.5

Iteration No: 193 ended. Search finished for the next optimal point.
Time taken: 7.5063
Function value obtained: -0.0283
Current minimum: -0.0283
Iteration No: 194 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7875387486518768, 'beta': 0.24600443845957412}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 3496 (100.0%) in 4.53 sec. Users per second: 771
SearchBayesianSkopt: Config 193 is suboptimal. Config: {'alpha': 0.7875387486518768, 'beta': 0.24600443845957412} - results: PRECISION: 0.0786041, PRECISION_RECALL_MIN_DEN: 0.0786041, RECALL: 0.0088813, MAP: 0.0279207, MAP_MIN_DEN: 0.0279207, MRR: 0.1949670, NDCG: 0.0792325, F1: 0.0159594, HIT_RATE: 0.5557780, ARHR_ALL_HITS: 0.2335097, NOVELTY: 0.0056745, AVERAGE_POPULARITY: 0.4757822, DIVERSITY_MEAN_INTER_LIST: 0.9463195, DIVERSITY_HERFINDAHL: 0.9946049, COVERAGE_ITEM: 0.0612991, COVERAGE_ITEM_CORRECT: 0.0308987, COVERAGE_USER: 1.00000

Iteration No: 200 ended. Search finished for the next optimal point.
Time taken: 7.5000
Function value obtained: -0.0279
Current minimum: -0.0283
SearchBayesianSkopt: Search complete. Best config is 182: {'alpha': 0.959079153887816, 'beta': 0.14131467903868594}



In [13]:
recommenders = [similarity_hybrid, IALS_recommender]
new_hybrid = MergeTwoModelsByScores(URM_stacked, recommenders)
new_hybrid.fit(alpha=0.759079153887816,beta=0.14131467903868594)

In [2]:
recommenders = [similarity_hybrid, IALS_recommender]
new_hybrid2 = MergeTwoModelsByScores(URM_stacked, recommenders)
new_hybrid2.fit(alpha=0.959079153887816,beta=0.14131467903868594)

NameError: name 'similarity_hybrid' is not defined

In [14]:
submission = create_submission(new_hybrid)

100%|████████████████████████████████████▉| 18056/18059 [15:36<00:00, 19.27it/s]


In [15]:
write_submission(submission, 'RP3hybridSLIM_IALS_stacked_LongTail3')